# Webscrape Vivino website for ratings

This notebook uses an iterative approach to explore scraping of www.vivino.com for red wine ratings data

In [10]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
import json
# Apply to all pages
import time
import random
import csv

params = {
    'country_code': '',
    'currency_code': '',
    'grape_filter': '',
    'merchant_id': '',
    'min_rating': '',
    'order_by': 'ratings_average',
    'order': 'desc',
    'page': '18820',
    'price_range_max': '',
    'price_range_min': '',
    'wine_type_ids[]': '1',
}

url = 'https://www.vivino.com/api/explore/explore'
userhead={'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.88 Safari/537.36'}
r = requests.get(url, params=params, headers=userhead)

In [115]:
json_data = r.json()
winedic = {}
winedic['name'] = []
winedic['region'] = []
winedic['country'] = []
winedic['num_reviews'] = []
winedic['rating'] = []
winedic['acidity'] = []
winedic['fizziness'] = []
winedic['intensity'] = []
winedic['sweetness'] = []
winedic['tannin'] = []
winedic['varietal'] = []
winedic['body'] = []

for wine in json_data['explore_vintage']['matches']:
    winedic['name'].append(wine['vintage']['name'])
    try:
        winedic['region'].append(wine['vintage']['wine']['region']['name'])
        winedic['country'].append(wine['vintage']['wine']['region']['country']['name'])
    except:
        winedic['region'].append('')
        winedic['country'].append('')
    
    try:
        winedic['num_reviews'].append(str(wine['vintage']['statistics']['ratings_count']))
        winedic['rating'].append(str(wine['vintage']['statistics']['ratings_average']))
    except: 
        winedic['num_reviews'].append('')
        winedic['rating'].append('')
    try:
        winedic['acidity'].append(str(wine['vintage']['wine']['taste']['structure']['acidity']))
        winedic['fizziness'].append(str(wine['vintage']['wine']['taste']['structure']['fizziness']))
        winedic['intensity'].append(str(wine['vintage']['wine']['taste']['structure']['intensity']))
        winedic['sweetness'].append(str(wine['vintage']['wine']['taste']['structure']['sweetness']))
        winedic['tannin'].append(str(wine['vintage']['wine']['taste']['structure']['tannin']))
    except:
        winedic['acidity'].append('')
        winedic['fizziness'].append('')
        winedic['intensity'].append('')
        winedic['sweetness'].append('')
        winedic['tannin'].append('')
        
    try:
        winedic['varietal'].append(wine['vintage']['wine']['style']['varietal_name'])
        winedic['body'].append(wine['vintage']['wine']['style']['body_description'])
    except:
        winedic['varietal'].append('')
        winedic['body'].append('')
r.close()

In [8]:
import requests
import pandas as pd
import numpy as np
import json
import time
import random
import csv

def scrape_single_page(json_data, csvwriter):
    for wine in json_data['explore_vintage']['matches']:
        winedic = {}
        winedic['name']=wine['vintage']['name']
        try:
            winedic['region']=wine['vintage']['wine']['region']['name']
            winedic['country']=wine['vintage']['wine']['region']['country']['name']
        except:
            winedic['region']=''
            winedic['country']=''

        try:
            winedic['num_reviews']=str(wine['vintage']['statistics']['ratings_count'])
            winedic['rating']=str(wine['vintage']['statistics']['ratings_average'])
        except: 
            winedic['num_reviews']=''
            winedic['rating']=''
        try:
            winedic['acidity']=str(wine['vintage']['wine']['taste']['structure']['acidity'])
            winedic['fizziness']=str(wine['vintage']['wine']['taste']['structure']['fizziness'])
            winedic['intensity']=str(wine['vintage']['wine']['taste']['structure']['intensity'])
            winedic['sweetness']=str(wine['vintage']['wine']['taste']['structure']['sweetness'])
            winedic['tannin']=str(wine['vintage']['wine']['taste']['structure']['tannin'])
        except:
            winedic['acidity']=''
            winedic['fizziness']=''
            winedic['intensity']=''
            winedic['sweetness']=''
            winedic['tannin']=''

        try:
            winedic['varietal']=wine['vintage']['wine']['style']['varietal_name']
            winedic['body']=wine['vintage']['wine']['style']['body_description']
        except:
            winedic['varietal']=''
            winedic['body']=''
        
        csvwriter.writerow(winedic.values())  


# Webscrape LCBO for red wines      
params = {
    'country_code': '',
    'currency_code': '',
    'grape_filter': '',
    'merchant_id': '',
    'min_rating': '',
    'order_by': 'ratings_average',
    'order': 'desc',
    'page': '2',
    'price_range_max': '',
    'price_range_min': '',
    'wine_type_ids[]': '1',
}

url = 'https://www.vivino.com/api/explore/explore'
userhead={'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.88 Safari/537.36'}
r = requests.get(url, params=params, headers=userhead)
json_data = r.json()
num_bottles = json_data['explore_vintage']['records_matched']
num_pages = np.int(num_bottles/12)
r.close()

with open('vivino_redwine.csv', 'w', encoding='utf-8', newline='') as csvfile:
    bottle_writer = csv.writer(csvfile)
    bottle_writer.writerow(['name','region','country','num_reviews','rating',\
                            'acidity','fizziness','intensity','sweetness','tannin','varietal','body'])
    for i in range(0, num_pages):
        params['page'] = str(i+1)
        r = requests.get(url, params=params, headers=userhead)
        json_data = r.json()
        scrape_single_page(json_data, bottle_writer)
        r.close()
        # Random sleep to avoid getting banned from the server
        time.sleep(random.randint(1,2))
        print('Finished page: ' + str(i+1) + ' out of ' + str(num_pages))

Finished page: 1 out of 37651
Finished page: 2 out of 37651
Finished page: 3 out of 37651
Finished page: 4 out of 37651
Finished page: 5 out of 37651
Finished page: 6 out of 37651
Finished page: 7 out of 37651
Finished page: 8 out of 37651
Finished page: 9 out of 37651
Finished page: 10 out of 37651
Finished page: 11 out of 37651
Finished page: 12 out of 37651
Finished page: 13 out of 37651
Finished page: 14 out of 37651
Finished page: 15 out of 37651
Finished page: 16 out of 37651
Finished page: 17 out of 37651
Finished page: 18 out of 37651
Finished page: 19 out of 37651
Finished page: 20 out of 37651
Finished page: 21 out of 37651
Finished page: 22 out of 37651
Finished page: 23 out of 37651
Finished page: 24 out of 37651
Finished page: 25 out of 37651
Finished page: 26 out of 37651
Finished page: 27 out of 37651
Finished page: 28 out of 37651
Finished page: 29 out of 37651
Finished page: 30 out of 37651
Finished page: 31 out of 37651
Finished page: 32 out of 37651
Finished page: 33

Finished page: 261 out of 37651
Finished page: 262 out of 37651
Finished page: 263 out of 37651
Finished page: 264 out of 37651
Finished page: 265 out of 37651
Finished page: 266 out of 37651
Finished page: 267 out of 37651
Finished page: 268 out of 37651
Finished page: 269 out of 37651
Finished page: 270 out of 37651
Finished page: 271 out of 37651
Finished page: 272 out of 37651
Finished page: 273 out of 37651
Finished page: 274 out of 37651
Finished page: 275 out of 37651
Finished page: 276 out of 37651
Finished page: 277 out of 37651
Finished page: 278 out of 37651
Finished page: 279 out of 37651
Finished page: 280 out of 37651
Finished page: 281 out of 37651
Finished page: 282 out of 37651
Finished page: 283 out of 37651
Finished page: 284 out of 37651
Finished page: 285 out of 37651
Finished page: 286 out of 37651
Finished page: 287 out of 37651
Finished page: 288 out of 37651
Finished page: 289 out of 37651
Finished page: 290 out of 37651
Finished page: 291 out of 37651
Finished

Finished page: 518 out of 37651
Finished page: 519 out of 37651
Finished page: 520 out of 37651
Finished page: 521 out of 37651
Finished page: 522 out of 37651
Finished page: 523 out of 37651
Finished page: 524 out of 37651
Finished page: 525 out of 37651
Finished page: 526 out of 37651
Finished page: 527 out of 37651
Finished page: 528 out of 37651
Finished page: 529 out of 37651
Finished page: 530 out of 37651
Finished page: 531 out of 37651
Finished page: 532 out of 37651
Finished page: 533 out of 37651
Finished page: 534 out of 37651
Finished page: 535 out of 37651
Finished page: 536 out of 37651
Finished page: 537 out of 37651
Finished page: 538 out of 37651
Finished page: 539 out of 37651
Finished page: 540 out of 37651
Finished page: 541 out of 37651
Finished page: 542 out of 37651
Finished page: 543 out of 37651
Finished page: 544 out of 37651
Finished page: 545 out of 37651
Finished page: 546 out of 37651
Finished page: 547 out of 37651
Finished page: 548 out of 37651
Finished

Finished page: 775 out of 37651
Finished page: 776 out of 37651
Finished page: 777 out of 37651
Finished page: 778 out of 37651
Finished page: 779 out of 37651
Finished page: 780 out of 37651
Finished page: 781 out of 37651
Finished page: 782 out of 37651
Finished page: 783 out of 37651
Finished page: 784 out of 37651
Finished page: 785 out of 37651
Finished page: 786 out of 37651
Finished page: 787 out of 37651
Finished page: 788 out of 37651
Finished page: 789 out of 37651
Finished page: 790 out of 37651
Finished page: 791 out of 37651
Finished page: 792 out of 37651
Finished page: 793 out of 37651
Finished page: 794 out of 37651
Finished page: 795 out of 37651
Finished page: 796 out of 37651
Finished page: 797 out of 37651
Finished page: 798 out of 37651
Finished page: 799 out of 37651
Finished page: 800 out of 37651
Finished page: 801 out of 37651
Finished page: 802 out of 37651
Finished page: 803 out of 37651
Finished page: 804 out of 37651
Finished page: 805 out of 37651
Finished

Finished page: 1031 out of 37651
Finished page: 1032 out of 37651
Finished page: 1033 out of 37651
Finished page: 1034 out of 37651
Finished page: 1035 out of 37651
Finished page: 1036 out of 37651
Finished page: 1037 out of 37651
Finished page: 1038 out of 37651
Finished page: 1039 out of 37651
Finished page: 1040 out of 37651
Finished page: 1041 out of 37651
Finished page: 1042 out of 37651
Finished page: 1043 out of 37651
Finished page: 1044 out of 37651
Finished page: 1045 out of 37651
Finished page: 1046 out of 37651
Finished page: 1047 out of 37651
Finished page: 1048 out of 37651
Finished page: 1049 out of 37651
Finished page: 1050 out of 37651
Finished page: 1051 out of 37651
Finished page: 1052 out of 37651
Finished page: 1053 out of 37651
Finished page: 1054 out of 37651
Finished page: 1055 out of 37651
Finished page: 1056 out of 37651
Finished page: 1057 out of 37651
Finished page: 1058 out of 37651
Finished page: 1059 out of 37651
Finished page: 1060 out of 37651
Finished p

...

Finished page: 36506 out of 37651
Finished page: 36507 out of 37651
Finished page: 36508 out of 37651
Finished page: 36509 out of 37651
Finished page: 36510 out of 37651
Finished page: 36511 out of 37651
Finished page: 36512 out of 37651
Finished page: 36513 out of 37651
Finished page: 36514 out of 37651
Finished page: 36515 out of 37651
Finished page: 36516 out of 37651
Finished page: 36517 out of 37651
Finished page: 36518 out of 37651
Finished page: 36519 out of 37651
Finished page: 36520 out of 37651
Finished page: 36521 out of 37651
Finished page: 36522 out of 37651
Finished page: 36523 out of 37651
Finished page: 36524 out of 37651
Finished page: 36525 out of 37651
Finished page: 36526 out of 37651
Finished page: 36527 out of 37651
Finished page: 36528 out of 37651
Finished page: 36529 out of 37651
Finished page: 36530 out of 37651
Finished page: 36531 out of 37651
Finished page: 36532 out of 37651
Finished page: 36533 out of 37651
Finished page: 36534 out of 37651
Finished page:

Finished page: 36747 out of 37651
Finished page: 36748 out of 37651
Finished page: 36749 out of 37651
Finished page: 36750 out of 37651
Finished page: 36751 out of 37651
Finished page: 36752 out of 37651
Finished page: 36753 out of 37651
Finished page: 36754 out of 37651
Finished page: 36755 out of 37651
Finished page: 36756 out of 37651
Finished page: 36757 out of 37651
Finished page: 36758 out of 37651
Finished page: 36759 out of 37651
Finished page: 36760 out of 37651
Finished page: 36761 out of 37651
Finished page: 36762 out of 37651
Finished page: 36763 out of 37651
Finished page: 36764 out of 37651
Finished page: 36765 out of 37651
Finished page: 36766 out of 37651
Finished page: 36767 out of 37651
Finished page: 36768 out of 37651
Finished page: 36769 out of 37651
Finished page: 36770 out of 37651
Finished page: 36771 out of 37651
Finished page: 36772 out of 37651
Finished page: 36773 out of 37651
Finished page: 36774 out of 37651
Finished page: 36775 out of 37651
Finished page:

Finished page: 36988 out of 37651
Finished page: 36989 out of 37651
Finished page: 36990 out of 37651
Finished page: 36991 out of 37651
Finished page: 36992 out of 37651
Finished page: 36993 out of 37651
Finished page: 36994 out of 37651
Finished page: 36995 out of 37651
Finished page: 36996 out of 37651
Finished page: 36997 out of 37651
Finished page: 36998 out of 37651
Finished page: 36999 out of 37651
Finished page: 37000 out of 37651
Finished page: 37001 out of 37651
Finished page: 37002 out of 37651
Finished page: 37003 out of 37651
Finished page: 37004 out of 37651
Finished page: 37005 out of 37651
Finished page: 37006 out of 37651
Finished page: 37007 out of 37651
Finished page: 37008 out of 37651
Finished page: 37009 out of 37651
Finished page: 37010 out of 37651
Finished page: 37011 out of 37651
Finished page: 37012 out of 37651
Finished page: 37013 out of 37651
Finished page: 37014 out of 37651
Finished page: 37015 out of 37651
Finished page: 37016 out of 37651
Finished page:

Finished page: 37229 out of 37651
Finished page: 37230 out of 37651
Finished page: 37231 out of 37651
Finished page: 37232 out of 37651
Finished page: 37233 out of 37651
Finished page: 37234 out of 37651
Finished page: 37235 out of 37651
Finished page: 37236 out of 37651
Finished page: 37237 out of 37651
Finished page: 37238 out of 37651
Finished page: 37239 out of 37651
Finished page: 37240 out of 37651
Finished page: 37241 out of 37651
Finished page: 37242 out of 37651
Finished page: 37243 out of 37651
Finished page: 37244 out of 37651
Finished page: 37245 out of 37651
Finished page: 37246 out of 37651
Finished page: 37247 out of 37651
Finished page: 37248 out of 37651
Finished page: 37249 out of 37651
Finished page: 37250 out of 37651
Finished page: 37251 out of 37651
Finished page: 37252 out of 37651
Finished page: 37253 out of 37651
Finished page: 37254 out of 37651
Finished page: 37255 out of 37651
Finished page: 37256 out of 37651
Finished page: 37257 out of 37651
Finished page:

Finished page: 37470 out of 37651
Finished page: 37471 out of 37651
Finished page: 37472 out of 37651
Finished page: 37473 out of 37651
Finished page: 37474 out of 37651
Finished page: 37475 out of 37651
Finished page: 37476 out of 37651
Finished page: 37477 out of 37651
Finished page: 37478 out of 37651
Finished page: 37479 out of 37651
Finished page: 37480 out of 37651
Finished page: 37481 out of 37651
Finished page: 37482 out of 37651
Finished page: 37483 out of 37651
Finished page: 37484 out of 37651
Finished page: 37485 out of 37651
Finished page: 37486 out of 37651
Finished page: 37487 out of 37651
Finished page: 37488 out of 37651
Finished page: 37489 out of 37651
Finished page: 37490 out of 37651
Finished page: 37491 out of 37651
Finished page: 37492 out of 37651
Finished page: 37493 out of 37651
Finished page: 37494 out of 37651
Finished page: 37495 out of 37651
Finished page: 37496 out of 37651
Finished page: 37497 out of 37651
Finished page: 37498 out of 37651
Finished page:

In [5]:
r.close()

In [11]:
json_data = r.json()
for wine in json_data['explore_vintage']['matches']:
    print('\n'+ wine['vintage']['name'])
    try:
        print('\t' + wine['vintage']['wine']['region']['name'])
        print('\t' + wine['vintage']['wine']['region']['country']['name'])
    except:
        print('Region: N/A')
        print('Country: N/A')
    
    try:
        print('\t' + str(wine['vintage']['statistics']['ratings_count']))
        print('\t' + str(wine['vintage']['statistics']['ratings_average']))
    except: 
        print('Review count: N/A')
        print('Rating: N/A')
    try:
        print('\t' + str(wine['vintage']['wine']['taste']['structure']['acidity']))
        print('\t' + str(wine['vintage']['wine']['taste']['structure']['fizziness']))
        print('\t' + str(wine['vintage']['wine']['taste']['structure']['intensity']))
        print('\t' + str(wine['vintage']['wine']['taste']['structure']['sweetness']))
        print('\t' + str(wine['vintage']['wine']['taste']['structure']['tannin']))
    except:
        print('Acidity: N/A')
        print('Fizziness: N/A')
        print('Intensity: N/A')
        print('Sweetness: N/A')
        print('Tannin: N/A')
        
    try:
        print('\t' + wine['vintage']['wine']['style']['varietal_name'])
        print('\t' + wine['vintage']['wine']['style']['body_description'])
    except:
        print('Varietal: N/A')
        print('Body description: N/A')


Odette Reserve Cabernet Sauvignon 2012
	Stags Leap District
	United States
	41
	4.7
	3.4717617
	None
	4.8784857
	1.4984864
	3.4532256
	Cabernet Sauvignon
	Very full-bodied

Pian dell'Orino Bassolino di Sopra Brunello di Montalcino 2010
	Brunello di Montalcino
	Italy
	41
	4.7
	3.7835555
	None
	2.7322223
	1.7655555
	3.5706668
	Brunello
	Very full-bodied

Domaine de La Romanée-Conti Corton Grand Cru 2016
	Corton Grand Cru
	France
	41
	4.7
	3.9159982
	None
	2.9049854
	1.6355115
	2.5055811
	Côte de Beaune Red
	Light-bodied

Falcor Le Bijou 2009
	Napa Valley
	United States
	41
	4.7
	3.9995208
	None
	4.9640718
	1.1798203
	3.4579644
	Bordeaux Blend
	Full-bodied

Del Dotto Cabernet Sauvignon St. Helena Mountain Family Reserve 2011
	St. Helena
	United States
	40
	4.7
	3.531501
	None
	4.915496
	1.4231727
	3.3990655
	Cabernet Sauvignon
	Very full-bodied

Gamble Cairo 2014
	Oakville
	United States
	40
	4.7
	3.9771836
	None
	4.9662857
	1.2876327
	3.345796
	Red Blend
	Full-bodied

Big Head The Bigge